
# W08 - Rule-Based Classification using PRISM Algorithm (from scratch)
**Name:** Collin Joseph  
**NIM:** 0706022310053  
**Topic:** Rule-Based Classification using PRISM Algorithm  
**Dataset:** Wine Dataset (from sklearn)

---

### Learning Objectives
- Apply the PRISM rule-based classification algorithm.  
- Interpret rules and explain model predictions in a human-understandable way.  
- Evaluate model performance using confusion matrix and classification report.

---


In [ ]:

# ===============================================================
# W08 - PRISM Algorithm from Scratch - Collin Joseph
# ===============================================================

import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# -----------------------------
# 1. Load Dataset
# -----------------------------
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Series(wine.target, name='Y')
df = pd.concat([X, y], axis=1)

print("Dataset Shape:", df.shape)
display(df.head())

# -----------------------------
# 2. Discretize Continuous Attributes (Low, Med, High)
# -----------------------------
def discretize_terciles(df, cols):
    df_disc = df.copy()
    for c in cols:
        q = df[c].quantile([0, 1/3, 2/3, 1]).values
        edges = np.unique(q)
        if len(edges) < 4:
            edges = np.linspace(df[c].min(), df[c].max(), 4)
        df_disc[c + '_binned'] = pd.cut(df[c], bins=edges, labels=['Low','Med','High'], include_lowest=True)
    return df_disc

df_disc = discretize_terciles(df, wine.feature_names)
binned_cols = [c + '_binned' for c in wine.feature_names]

# -----------------------------
# 3. Visualizations (EDA)
# -----------------------------
plt.figure(figsize=(8,4))
for cls in sorted(df['Y'].unique()):
    plt.hist(df[df['Y']==cls]['alcohol'], bins=12, alpha=0.6, label=f'class {cls}')
plt.xlabel('Alcohol')
plt.ylabel('Count')
plt.title('Histogram of Alcohol by Class')
plt.legend()
plt.show()

plt.figure(figsize=(6,5))
for cls in sorted(df['Y'].unique()):
    sel = df[df['Y']==cls]
    plt.scatter(sel['alcohol'], sel['color_intensity'], alpha=0.7, label=f'class {cls}')
plt.xlabel('Alcohol')
plt.ylabel('Color Intensity')
plt.title('Alcohol vs Color Intensity')
plt.legend()
plt.show()

# -----------------------------
# 4. PRISM Implementation from Scratch
# -----------------------------
class PrismFromScratch:
    def __init__(self, df, target_col, feature_cols):
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.feature_cols = feature_cols
        self.rules = {}
        
    def _accuracy_of_condition(self, data, attr, val, positive_class):
        subset = data[data[attr] == val]
        if len(subset) == 0:
            return 0, 0, 0.0
        p = (subset[self.target_col] == positive_class).sum()
        n = (subset[self.target_col] != positive_class).sum()
        acc = p / (p + n) if (p + n) > 0 else 0.0
        return int(p), int(n), float(acc)
    
    def learn_rules_for_class(self, positive_class):
        data = self.df.copy()
        rules_for_class = []
        positives_remaining = data[data[self.target_col] == positive_class]
        
        while len(positives_remaining) > 0:
            rule_conditions = {}
            covered = data.copy()
            while True:
                best = None
                best_metrics = (0,0,0.0)
                for attr in self.feature_cols:
                    if attr in rule_conditions:
                        continue
                    for val in data[attr].dropna().unique():
                        p, n, acc = self._accuracy_of_condition(covered, attr, val, positive_class)
                        if p == 0: continue
                        if (acc > best_metrics[2]) or (acc == best_metrics[2] and p > best_metrics[0]):
                            best = (attr, val)
                            best_metrics = (p, n, acc)
                if best is None: break
                attr, val = best
                rule_conditions[attr] = val
                covered = covered[covered[attr] == val]
                if all(covered[self.target_col] == positive_class): break
                if len(covered) == 0:
                    rule_conditions.pop(attr, None)
                    break
            if len(rule_conditions) == 0: break
            covered_set = data.copy()
            for a,v in rule_conditions.items():
                covered_set = covered_set[covered_set[a] == v]
            support = int((covered_set[self.target_col] == positive_class).sum())
            coverage = len(covered_set)
            rules_for_class.append({'conditions': rule_conditions.copy(), 'support': support, 'coverage': coverage})
            covered_pos_idx = covered_set[(covered_set[self.target_col] == positive_class)].index
            positives_remaining = positives_remaining.drop(index=covered_pos_idx, errors='ignore')
            data = data.drop(index=covered_pos_idx, errors='ignore')
            if (data[self.target_col] == positive_class).sum() == 0:
                break
        self.rules[positive_class] = rules_for_class
        return rules_for_class
    
    def fit(self):
        for c in sorted(self.df[self.target_col].unique()):
            self.learn_rules_for_class(c)
    
    def predict_instance(self, x):
        for c, rules in self.rules.items():
            for r in rules:
                conds = r['conditions']
                if all(x[a] == v for a,v in conds.items() if a in x):
                    return c
        return int(self.df[self.target_col].mode()[0])
    
    def predict(self, X_df):
        return np.array([self.predict_instance(row) for _, row in X_df.iterrows()])

# -----------------------------
# 5. Train-Test Split & Model Training
# -----------------------------
X_binned = df_disc[binned_cols]
y_col = df_disc['Y']
X_train, X_test, y_train, y_test = train_test_split(X_binned, y_col, test_size=0.3, random_state=42, stratify=y_col)
train_df = pd.concat([X_train, y_train], axis=1)

prism = PrismFromScratch(train_df, target_col='Y', feature_cols=binned_cols)
prism.fit()

print("=== Learned PRISM Rules ===")
for cls, rules in prism.rules.items():
    print(f"\nClass {cls}:")
    for i, r in enumerate(rules, 1):
        conds = " AND ".join([f"{a}={v}" for a,v in r['conditions'].items()])
        print(f" Rule {i}: IF {conds} THEN class {cls} (support={r['support']}, coverage={r['coverage']})")

# -----------------------------
# 6. Evaluation
# -----------------------------
y_pred = prism.predict(X_test)
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# -----------------------------
# 7. Predict New Data Points
# -----------------------------
new_data = pd.DataFrame({
    'alcohol': [14.0, 14.0],
    'malic_acid': [2.0, 2.0],
    'ash': [2.3, 2.2],
    'alcalinity_of_ash': [19.0, 11.0],
    'magnesium': [95.0, 95.0],
    'total_phenols': [2.2, 2.5],
    'flavanoids': [0.14, 0.5],
    'nonflavanoid_phenols': [0.14, 0.5],
    'proanthocyanins': [1.6, 1.5],
    'color_intensity': [7.0, 6.0],
    'hue': [0.7, 0.6],
    'od280/od315_of_diluted_wines': [3.2, 3.0],
    'proline': [550.0, 1400.0]
})
new_disc = discretize_terciles(new_data, wine.feature_names)[0][binned_cols]
preds = prism.predict(new_disc)
print("\nPredictions for New Data:")
print(preds)
